# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [25]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [26]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'BreastCancer-Experiment'

experiment=Experiment(ws, experiment_name)


# choose a name for experiment

project_folder = './breastcancer'
experiment

Name,Workspace,Report Page,Docs Page
BreastCancer-Experiment,quick-starts-ws-251498,Link to Azure Machine Learning studio,Link to Documentation


In [27]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "azurecapstone-project"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 5)



Found existing cluster, use it.


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

{'uci_id': 17, 'name': 'Breast Cancer Wisconsin (Diagnostic)', 'repository_url': 'https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic', 'data_url': 'https://archive.ics.uci.edu/static/public/17/data.csv', 'abstract': 'Diagnostic Wisconsin Breast Cancer Database.', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 569, 'num_features': 30, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['Diagnosis'], 'index_col': ['ID'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1993, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5DW2B', 'creators': ['William Wolberg', 'Olvi Mangasarian', 'Nick Street', 'W. Street'], 'intro_paper': {'title': 'Nuclear feature extraction for breast tumor diagnosis', 'authors': 'W. Street, W. Wolberg, O. Mangasarian', 'published_in': 'Electronic imaging', 'year': 1993, 'url': 'https://www.semanticscholar.org/paper/53

In [28]:
# NOTE: update the key to match the dataset name
found = False
key = "BreastData3"
description_text = "Bank Marketing DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/samfrost23/nd00333-capstone/master/starter_file/breast-cancer.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        ##from train import clean_data
        #import pandas as pd
        #data = TabularDatasetFactory.from_delimited_files("https://archive.ics.uci.edu/static/public/17/data.csv")
        #x, y = clean_data(data)
        #data_joined = pd.concat([x, y], axis=1)
        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.head(5)
df.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [36]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 5,
    "primary_metric" : 'AUC_weighted'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="diagnosis",   
                             path = project_folder,
                             enable_early_stopping= True,
                             #enable_onnx_compatible_models=True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )


In [37]:
# Submit experiment
from azureml.widgets import RunDetails
remote_run = experiment.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
BreastCancer-Experiment,AutoML_c758c99e-8bff-4ff0-a0dc-282052e4873b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
BreastCancer-Experiment,AutoML_c758c99e-8bff-4ff0-a0dc-282052e4873b,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

{'runId': 'AutoML_c758c99e-8bff-4ff0-a0dc-282052e4873b',
 'target': 'azurecapstone-project',
 'status': 'Completed',
 'startTimeUtc': '2024-02-03T12:28:42.166646Z',
 'endTimeUtc': '2024-02-03T12:56:55.597887Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 20 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'azurecapstone-project',
  'AMLSettingsJsonString': '{"path":null,"name":"BreastCancer-Experiment","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-251498","workspace_name":"quick-starts-ws-251498","region":"westeurope","compute_target":"azurecapstone-project","spark_service":null,"azure_service":"remote","many_models":fal

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [38]:
automl_best_run, automl_best_model = remote_run.get_output()
automl_best_run


automl_best_model


Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

0.9823785126533148

In [39]:
metrics = automl_best_run.get_metrics()
metrics['accuracy']

0.9823785126533148

In [40]:
#TODO: Save the best model
import joblib
from azureml.core.model import Model

description = "Best-fit AutoML model trained on Breast Cancer dataset"

os.makedirs('outputs', exist_ok=True)
joblib.dump(automl_best_model, filename="outputs/automl-model.pkl")
automl_model = remote_run.register_model(model_name='automl-breastcancer', description=description)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [41]:
from azureml.core.webservice import AciWebservice

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    description='Predict tumor diagnosis - Malignant(1) or Benign(0)',
    auth_enabled=True
)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [43]:

from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.automl.core.shared import constants

model = Model(ws, 'automl-breastcancer')


myenv = automl_best_run.get_environment()
entry_script = 'predict.py'
#best_run.download_file('outputs/scoring_file_v_1_0_0.py', entry_script)
#best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'myenv.yml')
inference_config = InferenceConfig(entry_script=entry_script, environment=myenv)

service = Model.deploy(workspace=ws, 
                       name='automl-breastcancer', 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)



Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-02-03 13:07:12+00:00 Creating Container Registry if not exists..
2024-02-03 13:17:12+00:00 Registering the environment..
2024-02-03 13:17:12+00:00 Use the existing image.
2024-02-03 13:17:13+00:00 Submitting deployment to compute..
2024-02-03 13:17:19+00:00 Checking the status of deployment automl-breastcancer..
2024-02-03 13:20:07+00:00 Checking the status of inference endpoint automl-breastcancer.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [47]:
if service.enable_app_insights==False:
    service.update(enable_app_insights=True)
print(service.get_logs())

2024-02-03T13:26:01,282972412+00:00 - rsyslog/run 
2024-02-03T13:26:01,292385512+00:00 - gunicorn/run 
2024-02-03T13:26:01,296626512+00:00 | gunicorn/run | 
2024-02-03T13:26:01,299956612+00:00 | gunicorn/run | ###############################################
2024-02-03T13:26:01,305427012+00:00 | gunicorn/run | AzureML Container Runtime Information
2024-02-03T13:26:01,308990412+00:00 | gunicorn/run | ###############################################
2024-02-03T13:26:01,318880412+00:00 | gunicorn/run | 
2024-02-03T13:26:01,321600512+00:00 - nginx/run 
2024-02-03T13:26:01,340423912+00:00 | gunicorn/run | 
2024-02-03T13:26:01,353638712+00:00 | gunicorn/run | AzureML image information: openmpi4.1.0-ubuntu20.04, Materializaton Build:20230628.v2
2024-02-03T13:26:01,357401212+00:00 | gunicorn/run | 
2024-02-03T13:26:01,369929312+00:00 | gunicorn/run | 
2024-02-03T13:26:01,378080012+00:00 | gunicorn/run | PATH environment variable: /azureml-envs/azureml-automl/bin:/opt/miniconda/bin:/usr/local/sbi

In [48]:
print("State: "+service.state)
print("Scoring URI: "+service.scoring_uri)

State: Healthy
Scoring URI: http://87cbc28d-3a5b-4690-9955-058d8758c3c3.westeurope.azurecontainer.io/score


**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
